<a href="https://colab.research.google.com/github/risyadmaulana3332180024/Skripsi-/blob/main/RiceVarietyCNN_VGG16Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pengenalan**

Beras merupakan suatu komoditas yang strategis karena menjadi bahan pangan pokok bangsa Indonesia. Konsumsi beras setiap tahun selalu meningkat seiring dengan laju penambahan jumlah penduduk.

Pentingnya identifikasi jenis beras sangat perlu dilakukan untuk mengetahui kandungan gizi serta nutrisi yang terkandung dalam bahan pokok makanan ini sesuai dengan kebutuhan yang dibutuhkan.


Penelitian ini memiliki tujuan untuk melakukan proses analisa identifikasi jenis beras dengan bantuan pengolahan citra digital menggunakan metode convolutional neural network. Menggunakan Google Collabs, data citra gambar beras akan dianalisa dan dilatih untuk melakukan proses identifikasi jenis beras.


# **Memasukan Library Yang Dibutuhkan**

Untuk Model : TensorFlow dan Keras

In [ ]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
import tensorflow as tf
from tensorflow import keras
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers.core import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam
from keras.models import Model
import cv2

#display model to PNG / FLOW of the model architecture
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from google.colab.patches import cv2_imshow

# **Memasukan *Library* Untuk *Transfer Learning***

Menggunakan *Learning* Dari *VGG16Net*


In [ ]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.resnet import ResNet50
from keras.applications.inception_v3 import InceptionV3 
from keras.applications.inception_resnet_v2 import InceptionResNetV2 
from keras.applications.densenet import DenseNet121

# **Memasukan library Untuk Labelisasi**

Menggunakan : Numpy, Sklearn, PIL, Os, Imutils, Argparse dan Matplotlib

In [ ]:
from sklearn.preprocessing import LabelBinarizer # untuk binarisasi label 
from sklearn.model_selection import train_test_split # untuk membagi data train dan data test 
from sklearn.metrics import classification_report # performance measurement , untuk menghasilkan confusion matrix

# untuk load data 
from PIL import Image 
from imutils import paths
import numpy as np
import argparse
import os
import matplotlib.pyplot as plt #untuk menampilkan contoh data dalam gambar

# **Menghubungkan Ke Penyimpanan Drive**

Menggunakan *Google Drive* Sebagai Jalan Masuknya Data dan Penyimpanan Hasil

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **Labelisasi Berdasarkan Folder**

Melakukan Pembagian Labelisasi Dari Data Yang Masuk Berupa :

- Memasukan Data Dari Penyimpanan Yang Digunakan
- Merubah Ukuran Citra 
- Membagi Data Training dan Validasi






In [ ]:
print("Menampilkan Gambar")
#imagePaths = paths.list_images("/content/drive/MyDrive/RiceDataset")
imagePaths = paths.list_images("/content/drive/MyDrive/RiceDataset")
data = []
labels = []

# loop over our input images
for imagePath in imagePaths:
	#Resize citra menjadi ukuran 224*224
    #Bentuk citra ini dapat diubah ubah
	image = Image.open(imagePath)
	image = np.array(image.resize((224, 224))) / 255.0
	data.append(image)
	# extract the class label from the file path and update the
	# labels list
	label = imagePath.split(os.path.sep)[-2]
	# label = imagePath.title
	labels.append(label)
	# print(label)
print(label)


lb = LabelBinarizer()
labels = lb.fit_transform(labels)
#split dataset dan data train alokasi normal adalah 25%
(trainX, testX, trainY, testY) = train_test_split(np.array(data),
	np.array(labels), test_size=0.25) #komposisi data training 75%, validationnya 25%
tes_data = testX[90]
plt.imshow(tes_data)
print(trainY)

# **Transfer Learning Menggunakan VGG16Net**

Memanfaatkan model yang sudah dilatih terhadap suatu dataset.

- Model : VGG16Net
- Weight : ImageNet
- Optimazer : Adam

In [ ]:
#RUN FOR VGG-16
IMAGE_SIZE = [224,224] #size of images
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False) #initiation of import VGG16

#ADDING FOR CUSTOM LAYERS
for layer in vgg16.layers:
    layer.trainable = False

x = Flatten()(vgg16.output)
x = tf.keras.layers.Dense(1024, activation='relu')(x)


prediction = Dense(5, activation='softmax')(x) #initiation for number of maximal output dense
model = Model(inputs=vgg16.input, outputs=prediction)


model.compile(loss='categorical_crossentropy', #categorical_crosentropy is for multilabel (over than 2 classes)
                    optimizer=tf.optimizers.Adam(0.0001), #learning rate (estimation/itterate)
                    metrics=['accuracy'])

import IPython

SVG(model_to_dot(model).create(prog='dot', format='svg'))

keras.utils.plot_model(model, to_file='test_CNN_Keras.png', show_shapes=True)
#IPython.display.Image('test_keras_plot_model.png')

# **Menampilkan Hasil Model**



In [ ]:
model.summary() 

# **Tunning HyperParameter**

Melakukan Pengujian Model Dalam Mendapatkan Performa Terbaik

In [ ]:
# Training Model
print("[INFO] training network...")
opt = Adam(learning_rate=0.0001)
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
H = model.fit(trainX, trainY, validation_data=(testX, testY),epochs=50, batch_size=32)

# Evaluasi Model Architecture
print("[INFO] evaluating network...")

predictions = model.predict(testX, batch_size=2)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=lb.classes_))

#model.save("RiceClassificationVgg16.hdf5")
model.save("RiceClassificationVgg16.hdf5")

# **Menampilkan History Loss & Accuracy**

Menampilkan Hasil Kesalahan dan Akurasi Dari Model Yang Diuji

In [ ]:
H.history['loss']
print ("Menampilkan Akurasi")
H.history['accuracy']


# **Plot Grafik Loss dan Accuracy**

Menampilkan Grafik Dari Kesalahan Dari Model Yang Diuji Berdasarkan Banyaknya Iterasi Yang Diberikan

In [ ]:
# Plot history: Plot Loss
import matplotlib.pyplot as plt
plt.plot(H.history['loss'], label='Training Loss')
plt.plot(H.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.ylabel('value')
plt.xlabel('No. epoch')
plt.legend(loc="upper left")
plt.show()

Menampilkan Grafik Dari Akurasi Dari Model Yang Diuji Berdasarkan Banyaknya Iterasi Yang Diberikan

In [ ]:
# Plot history: Plot Acc
plt.plot(H.history['accuracy'], label='Training Accuracy')
plt.plot(H.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.ylabel('value')
plt.xlabel('No. epoch')
plt.legend(loc="upper left")
plt.show()

# **Persiapan Evaluasi Terhadap Model Yang Sudah Dilatih**



In [ ]:
import requests
from io import BytesIO

from PIL import Image
import numpy as np
from matplotlib import pyplot as plt

Deklarasi Label Yang Akan Digunakan

In [ ]:
# Parameters
input_size = (224,224)

#define input shape
channel = (3,)
input_shape = input_size + channel

#define labels
labels = ['Arborio', 'Basmati', 'Ipsala', 'Jasmine', 'karacadag']

In [ ]:
def preprocess(img,input_size):
    nimg = img.convert('RGB').resize(input_size, resample= 0)
    img_arr = (np.array(nimg))/255
    return img_arr

def reshape(imgs_arr):
    return np.stack(imgs_arr, axis=0)

Memasukan Model Path Prediksi Yang Telah di Training

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
MODEL_PATH ='RiceClassificationVgg16.hdf5'
model = load_model(MODEL_PATH,compile=False)


# **Melakukan Prediksi Menggunakan Data Test Dari Model Yang Sudah Dibuat**

In [ ]:
testPaths = paths.list_images("/content/drive/MyDrive/RiceDatatest/FullTest")
labels = ['Arborio', 'Basmati', 'Ipsala', 'Jasmine', 'Karacadag']
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

for imagePath in testPaths:
    image = Image.open(imagePath)
    plt.imshow(image)
    plt.show()
    image = np.array(image.resize((224, 224))) / 255.0
    image2 = np.expand_dims(image, axis=0)
    pred = model.predict(image2)
    print(pred)
    y_classes = pred.argmax(axis=-1)
    print(y_classes)
    print('Image '+ imagePath + ' Classify as: '+ str(lb.classes_[y_classes][0]))

**Evaluasi Dengan Coffusion Matrix**

In [ ]:
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix , classification_report
import numpy as np
import pandas as pd
import seaborn as sns


In [ ]:
def print_confusion_matrix(confusion_matrix, class_names, figsize = (10,7), fontsize=14):
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names, 
    )
    fig = plt.figure(figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('Truth')
    plt.xlabel('Prediction')

In [ ]:
truth =      ["A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K"]
prediction = ["J","B","I","J","K","K","B","I","B","K","A","B","I","I","K","A","B","I","I","K","A","B","I","B","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K","A","B","I","J","K"]

In [ ]:
cm = confusion_matrix(truth,prediction)
print_confusion_matrix(cm,["Arborio","Basmati","Ipsala","Jasmine","Karacadag"])


In [ ]:
print(classification_report(truth, prediction))